## Created nate_branch for phase 2 group project

## Final Project Submission

Please fill out:
* Student name: 
* Student pace: self paced / part time / full time
* Scheduled project review date/time: 
* Instructor name: 
* Blog post URL:


Possible stakeholder:
Here's how much your home will sell for. Ask Shane about standard average  asking price -  haggle price = closing price so we can adjust our recommendation up that percent 
say asking_price = 100,000 and buyer haggles to 90,000, we would factor that in and recommend asking_price be 110,000

object to datetime for this purpose. ['yr_renovated'] has 3842 Null values.

our price mean is 540296.5735055795

['price'] and ['zipcode'] have a very low correlation of -0.05340243437772487

Top correlations with price
sqft_living = 0.70
grade = 0.66
sqft_above = 0.60
sqft_living15 = 0.58


['yr_renovated'] has something in it that the second_model doesn't like

In [1]:
# Import the necessary libraries


import pandas as pd 
import matplotlib.pyplot as plt
import math
from math import gamma
import scipy as sp
import numpy as np
from matplotlib.lines import Line2D
import itertools
from scipy import stats
import scipy.stats as stats
from scipy.stats import norm
import statsmodels.api as sm
import seaborn as sns
from collections import Counter
import folium
from folium import plugins
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from sklearn.cluster import KMeans, AgglomerativeClustering

from statsmodels.formula.api import ols
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate, ShuffleSplit, RandomizedSearchCV
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn import datasets
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler, MaxAbsScaler, QuantileTransformer 
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV, Ridge, RidgeCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
#validation libraries
from IPython.display import display
from sklearn.metrics import mean_squared_error, mean_absolute_error

pd.set_option('display.max_columns', None)
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')
%config InlineBackend.figure_format = 'retina'
%matplotlib inline

ModuleNotFoundError: No module named 'folium'

In [ ]:
# read the data into a variable called data
data = pd.read_csv('data/kc_house_data.csv')

# make a copy of data to work with calle df
df = data.copy(deep=True)

# converted dates from object to datetime
df.date = pd.to_datetime(df.date,infer_datetime_format=True)
df['date']=df['date'].apply(lambda x: x.toordinal())

# getting rid of a question mark in one of the columns
df['sqft_basement'] = df['sqft_basement'].replace({'?':np.nan}).astype(float)
df.head(50)

# get rid of those pesky NaN
df['yr_renovated'] = df['yr_renovated'].fillna(0)

# convert to log_price 
df['log_price'] = np.log(df['price']) #$$$$$$

# removing normalized price from our log_price df
df.drop(['price'], axis=1, inplace=True) #$$$$$

df['view'] = df['view'].fillna(0)

# check log_price minimum
df.log_price.min()

In [ ]:
df.waterfront = df.waterfront.fillna(0)

df_dummies = pd.get_dummies(df, prefix='waterfront', prefix_sep='_',
                           columns=['waterfront'], drop_first = True)
dummy_waterfront = df_dummies['waterfront_1.0']


df_dummies['dummy_waterfront'] = dummy_waterfront

df_dummies.drop('waterfront_1.0', axis=1, inplace=True)

df_dummies.head()
#df.head()


## Adjusting Longitude and Latitude

In [ ]:
# Now to deal with longitude and latitude
# Round off after the decimal to help our model 
# df = df.round({"long":4, "lat":4})

# **A second option to deal with long and lat**
# Change long and lat to radians instead of degrees
df["long"] = np.radians(df["long"])
df["lat"] = np.radians(df["lat"])

## KMeans color clustering the map

In [ ]:
# k means
# https://towardsdatascience.com/visualizing-clusters-with-pythons-matplolib-35ae03d87489
kmeans = KMeans(n_clusters=3, random_state=0)
df['cluster'] = kmeans.fit_predict(df[['lat', 'long']])
# get centroids
centroids = kmeans.cluster_centers_
cen_x = [i[0] for i in centroids] 
cen_y = [i[1] for i in centroids]
## add to df
df['cen_x'] = df.cluster.map({0:cen_x[0], 1:cen_x[1], 2:cen_x[2]})
df['cen_y'] = df.cluster.map({0:cen_y[0], 1:cen_y[1], 2:cen_y[2]})
# define and map colors
colors = ['#DF2020', '#81DF20', '#2095DF']
df['c'] = df.cluster.map({0:colors[0], 1:colors[1], 2:colors[2]})

# Assigning our model variables 

In [ ]:
dropped_cols_for_work = [ 'id','sqft_lot', 'floors'
                ,'waterfront', 'yr_built','sqft_basement', 'date'
            ,'zipcode', 'yr_renovated', 'bathrooms', 'cen_x', 'cen_y']
X_train.waterfront.value_counts()

In [ ]:
y_reg = data.price
y = df.log_price  #changed log_price to log Sunday night 8:40
X = df.drop('log_price', axis=1) #changed log_price to log Sunday night 8:40

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2) # ,random_state=10) if you want the samples to stay the same each time

# X_train.drop(['date'], axis=1, inplace=True)
# X_test.drop(['date'], axis=1, inplace=True)

# # changed ? in ['sqft_basement to NaN']
# X_train['sqft_basement'] = X_train['sqft_basement'].replace({'?':np.nan}).astype(float)
# X_test['sqft_basement'] = X_test['sqft_basement'].replace({'?':np.nan}).astype(float)
X_train.tail(5)

## Plotting out the locations of our homes over a map of King Co.

In [ ]:
fig, ax = plt.subplots(figsize = (15,8))
sns.scatterplot(X_train.long, X_train.lat, alpha=.7);

plt.title('Longitude and Latitude Coordinates of Houses in King County')
plt.xlabel('Longitude')
plt.ylabel('latitude');

In [ ]:
plt.figure(figsize=(15,8))

plt.scatter(df.long, df.lat, c=df.c, alpha = 0.6, s=10,)

# create a list of legend elemntes
## markers / records
legend_elements = [Line2D([0], [0], marker='o', color='w', label='Cluster {}'.format(i+1), 
               markerfacecolor=mcolor, markersize=5) for i, mcolor in enumerate(colors)]

plt.legend(handles=legend_elements, loc='upper right');

## A heatmap to help inspect correlation

In [ ]:
# Create a df with the target as the first column,
# then compute the correlation matrix
heatmap_data = pd.concat([y_train, X_train], axis=1)
corr = heatmap_data.corr()

# Set up figure and axes
fig, ax = plt.subplots(figsize=(15, 15))

# Plot a heatmap of the correlation matrix, with both
# numbers and colors indicating the correlations
sns.heatmap(
    # Specifies the data to be plotted
    data=corr,
    # The mask means we only show half the values,
    # instead of showing duplicates. It's optional.
    mask=np.triu(np.ones_like(corr, dtype=bool)),
    # Specifies that we should use the existing axes
    ax=ax,
    # Specifies that we want labels, not just colors
    annot=True,
    # Customizes colorbar appearance
    cbar_kws={"label": "Correlation", "orientation": "horizontal", "pad": .2, "extend": "both"}
)

# Customize the plot appearance
ax.set_title("Heatmap of Correlation Between Attributes (Including Target)");

## Correlation amung the features

In [ ]:
# bedrooms
# sqft_living
# condition
# grade
# lat
# sqft_living15
# view
# sqft_above

In [ ]:
print(data.price.min())
print(data.price.max())

In [ ]:
fig, ax = plt.subplots(figsize=(8, 10))

ax.hist(y_reg, bins=100)

ax.set_xlabel("Price")
ax.set_ylabel("Amount of Homes at 'Price'")
ax.set_title("Distribution of King County House Prices");

In [ ]:
fig, ax = plt.subplots(figsize=(8, 10))

ax.hist(y_train, bins=100)

ax.set_xlabel("Log Price")
ax.set_ylabel("Amount of Homes at 'Log Price'")
ax.set_title("Distribution of King County House at Log Prices");

In [ ]:
plt.scatter(df['sqft_living'], data['price'])
plt.title('King County, WA. Real Estate Normalized Price')
plt.xlabel('Square Feet of Living Space Normal Price')
plt.ylabel('Price');

In [ ]:
plt.scatter(df['sqft_living'], df['log_price']) # changed log_price to price Sunday night 8:40
plt.title('King County, WA. Real Estate Logarithmic Price')
plt.xlabel('Square Feet of Living Space Log Price')
plt.ylabel('Price');

In [ ]:
most_correlated_feature = 'sqft_living'

In [ ]:
baseline_model = LinearRegression()

## Model number one

In [ ]:
splitter = ShuffleSplit(n_splits=3, test_size=0.25, random_state=0)

baseline_scores = cross_validate(
    estimator=baseline_model,
    X=X_train[[most_correlated_feature]],
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Train score:     ", baseline_scores["train_score"].mean())
print("Validation score:", baseline_scores["test_score"].mean())

##  Let's have a look at only numeric columns shall we?

In [ ]:
X_train_numeric = X_train.select_dtypes(include=['int64', 'float64'])

X_train_numeric.isnull().sum()

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=8, figsize=(18, 30))
fig.set_tight_layout(True)

for index, col in enumerate(X_train_numeric.columns):
    ax = axes[index//3][index%3]
    ax.scatter(X_train_numeric[col], y_train, alpha=0.2)
    ax.set_xlabel(col)
    ax.set_ylabel("listing price")

In [ ]:
X_train_second_model = X_train_numeric.drop(columns = [ 'id','sqft_lot', 'floors'
                                                       ,'waterfront', 'yr_built','sqft_basement', 'date'
                                                       ,'zipcode', 'yr_renovated', 'bathrooms', 'cen_x', 'cen_y'])
X_train_second_model.head()

# ****view has something in it that the model doesn't like****

# bedrooms
# sqft_living
# grade
# lat
# sqft_living15
#'sqft_above'
# condition



## Second model

In [ ]:
second_model = LinearRegression()

second_model_scores = cross_validate(
    estimator=second_model,
    X=X_train_second_model,
    y=y_train,
    return_train_score=True,
    cv=splitter
)

print("Current Model")
print("Train score:     ", second_model_scores["train_score"].mean())
print("Validation score:", second_model_scores["test_score"].mean())
print()
print("Baseline Model")
print("Train score:     ", baseline_scores["train_score"].mean())
print("Validation score:", baseline_scores["test_score"].mean())

Given the fact that the Train Scores and Validation Scores are relatively near to eachother, it may be said that there is very little multicollinearity in the second_model

## Tests from Sunday morning (after data loss debacle and recovery), July 11th

## Tests from Saturday night, July 10th

## Test with Lat and Long excluded

In [ ]:
# from sklearn.linear_model import LinearRegression
# linreg = LinearRegression()
# linreg.fit(X_train_second_model, y_train)

# y_hat_train = linreg.predict(X_train)
# y_hat_test = linreg.predict(X_test)

## OLS and the search for multicollinearity

In [ ]:
sm.OLS(y_train, sm.add_constant(X_train_second_model)).fit().summary()

In [ ]:
# Importances are based on coefficient magnitude, so
# we need to scale the data to normalize the coefficients
X_train_for_RFECV = StandardScaler().fit_transform(X_train_second_model)

model_for_RFECV = LinearRegression()

# Instantiate and fit the selector
selector = RFECV(model_for_RFECV, cv=splitter)
selector.fit(X_train_for_RFECV, y_train)

# Print the results
print("Was the column selected?")
for index, col in enumerate(X_train_second_model.columns):
    print(f"{col}: {selector.support_[index]}")

# Third Model

In [ ]:
# print(df[["bedrooms","sqft_living"]].corr()) # .57
# print(df[["bedrooms","condition"]].corr())# .02
# print(df[["bedrooms","grade"]].corr())# .35
# print(df[["bedrooms","lat"]].corr())# .009
# print(df[["bedrooms","sqft_living15"]].corr()) # .39
# print(df[["bedrooms","view"]].corr())# .07 
# print(df[["bedrooms","sqft_above"]].corr())# .47

# print(df[["sqft_living","condition"]].corr()) # .05
# print(df[["sqft_living","grade"]].corr()) # .76 ***********high
# print(df[["sqft_living","lat"]].corr()) # .05
# print(df[["sqft_living","sqft_living15"]].corr()) # .75 ********high
# print(df[["sqft_living","view"]].corr()) # .28 
# print(df[["sqft_living","sqft_above"]].corr()) # .87 ********high

# print(df[["condition","grade"]].corr()) # -.14
# print(df[["condition","lat"]].corr()) # -.015
# print(df[["condition","sqft_living15"]].corr()) # -.09
# print(df[["condition","view"]].corr()) # .04 
# print(df[["condition","sqft_above"]].corr()) # - .10

# print(df[["grade","lat"]].corr()) # .11
# print(df[["grade","sqft_living15"]].corr()) # .71 *****high
# print(df[["grade","view"]].corr()) # .24 
# print(df[["grade","sqft_above"]].corr()) # .75 ********high

# print(df[["lat","sqft_living15"]].corr()) # .04
# print(df[["lat","view"]].corr()) # .006 
# print(df[["lat","sqft_above"]].corr()) # -.001

# print(df[["sqft_living15","view"]].corr()) # .27 
# print(df[["sqft_living15","sqft_above"]].corr()) # .73********high

# sqft_living and waterfront are lower than sqft_living an view

## Selecting best features

In [ ]:
best_features = ['sqft_living', 'view', 'lat'] # minus condition
#bedrooms
# sqft_living
# grade
# lat
# sqft_living15
#'sqft_above'
# condition


In [ ]:
X_train_final = X_train[best_features]
X_test_final = X_test[best_features]

###  R - Squared 

In [ ]:
final_model = LinearRegression()

# fit the model on X_train_final and y_train
final_model.fit(X_train_final, y_train)

# Score the model on X_test_final and y_train
final_model.score(X_test_final, y_test)

### Mean Squared Error (MSE)

In [ ]:
mean_squared_error(y_test, final_model.predict(X_test_final), squared=False)

In [ ]:
# Run this cell without changes
print(pd.Series(final_model.coef_, index=X_train_final.columns, name="Coefficients"))
print()
print("Intercept:", final_model.intercept_)

### Investigating Linearity

In [ ]:
y_test.min()

In [ ]:
preds = final_model.predict(X_test_final) # commented out this line along with below change 
fig, ax = plt.subplots()

perfect_line = np.arange(y_test.min(), y_test.max())
ax.plot(perfect_line,linestyle="--", color="orange", label="Perfect Fit") # perfect_line, ***removed this from round bracket first position
ax.scatter(y_test, preds, alpha=0.5)
ax.set_xlabel("Actual Price")
ax.set_ylabel("Predicted Price")
ax.legend();

## QQ plot (Quantile-Quantile)

In [ ]:
residuals = (y_test - preds)
sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True);

### Variance Inflation Factor (Above 5 is bad for your health)

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
vif = [variance_inflation_factor(X_train_final.values, i) for i in range(X_train_final.shape[1])]
pd.Series(vif, index=X_train_final.columns, name="Variance Inflation Factor")

###  Homeoscedasticity Check

In [ ]:
fig, ax = plt.subplots()

ax.scatter(preds, residuals, alpha=0.5)
ax.plot(preds, [0 for i in range(len(X_test))])
ax.set_xlabel("Predicted Value")
ax.set_ylabel("Actual - Predicted Value");